In [ ]:
JsonDocumentStore
    """
    An implementation of a document store that uses JSON files to store the documents and
    maintains an index.json containing some basics of the documents
    """



TableDataStore
    """
    Stores data as a list of lists that can represent a table.

    This is a good store when you are capturing nested or tabular data.

    :param columns: a list of the column names (default to dynamic)
    :param rows: initial set of rows (default to empty)

    """
    

DictDataStore
    """
    Stores data as a list of dictionaries that can be any structure

    This is a good store when you are capturing nested or semi-structured data
    """
    

DataStoreHelper
    """
    A small helper that can convert a dictionary back into a store
    type
    """


